### 1. Análise Exploratória de Dados

In [ ]:
#importando as principais bibliotecas 

import pandas as pd 

import seaborn as sns 
import matplotlib.pyplot as plt

from shapely.geometry import Point
import geopandas as gpd
from geopandas.tools import sjoin

In [ ]:
#efetuando a importação dos dados

df_eleitoral = pd.read_csv(r'C:\Users\cesar\Downloads\projeto_eleicao_2022\perfil_eleitorado_2022\perfil_eleitor_secao_2022_BA.csv', sep=";", encoding="latin-1" )


In [ ]:
#visualizando as primeiras linhas

df_eleitoral.head(10)

In [ ]:
#verificando o número de linhas e colunas

df_eleitoral.shape

In [ ]:
#principais informações do df

df_eleitoral.info()

In [ ]:
#principais estatisticas descritivas do df

df_eleitoral.describe()

In [ ]:
#visualizando colunas no df

df_eleitoral.columns

In [ ]:
#criando função para visualizar os valores únicos e/ou nulos por colunas (ter o pandas importado)

def valores_unicos(df):

    for colunas in df.columns:
        print(f'Coluna {colunas}:')
        contar_valores = df[colunas].value_counts(dropna=False)
        porcentagem_valores = df[colunas].value_counts(dropna=False, normalize=True) * 100
        resultado_valores = pd.concat([contar_valores, porcentagem_valores], axis=1, 
                                     keys=['Qnt', 'Porcentagem'])
        print(resultado_valores)

        print(f'Valores nulos: {df[colunas].isnull().sum()}')
        print('\n')

valores_unicos(df_eleitoral)

In [ ]:
#criando função para gerar gráfico em barras para a visualização dos valores por coluna

def visualizar_barras (coluna, df):

    valores_totais = len(df[coluna])
    contador_valores = df[coluna].value_counts(dropna=False)
    porcentagem_valores = (contador_valores/valores_totais) * 100

    porcentagem_valores.plot(kind='bar')
    plt.title(f'Dados referentes a coluna: {coluna}')
    plt.xlabel(coluna)
    plt.ylabel('%')
    plt.show

In [ ]:
visualizar_barras('DS_ESTADO_CIVIL', df_eleitoral)

In [ ]:
visualizar_barras('DS_FAIXA_ETARIA', df_eleitoral)

In [ ]:
visualizar_barras('DS_GRAU_ESCOLARIDADE', df_eleitoral)

In [ ]:
visualizar_barras('DS_GENERO', df_eleitoral)

### 2. Tratando dataframe eleitoral

In [ ]:
#criando uma cópia do dataframe antes do tratamento 

df_tratado = df_eleitoral.copy()


In [ ]:
#tratando os valores nulos (#NULO, #NE, -1 ou -3, assim como descrito no material do TSE)

df_tratado.replace({'#NULO': None, '#NE': None, '-1': None, '-3': None}, inplace= True)

In [ ]:
df_tratado.head(5)

In [ ]:
#verificando o número de linhas do df_tratado

df_tratado.shape

### 3. Dados geoespaciais dos muninicipios da Bahia

In [ ]:
#visualizando dados do nosso arquivo SHP que contém dados geoespaciais da divisão político-administrativa da bahia

df_geoespacial = gpd.read_file(r'C:\Users\cesar\Downloads\projeto_eleicao_2022\DIVISAO_POLITICO_ADMINISTRATIVA_2017_06_14\DPA_A_100K_2017_06_14_GCS_SIR_SEI.shp')

print(df_geoespacial.head(5))

In [ ]:
df_geoespacial.columns

In [ ]:
valores_unicos(df_geoespacial)

In [ ]:
# Contar o número de municípios únicos em cada conjunto de dados
num_municipios_geoespacial = df_geoespacial['MUNICIPIO'].nunique()
num_municipios_eleitoral = df_eleitoral['NM_MUNICIPIO'].nunique()

print(num_municipios_eleitoral)
print(num_municipios_geoespacial)


In [ ]:
df_geoespacial.plot()

plt.show()

In [ ]:
#verificando o tipo de variáveis em cada coluna para unir os dois dataframes (df_mapa e df_tratado)

df_geoespacial.info()

In [ ]:
#Tratando os dataframes antes de os unir em um GeoDataFrame

df_tratado['NM_MUNICIPIO'] = df_tratado['NM_MUNICIPIO'].str.upper()
df_geoespacial['MUNICIPIO'] = df_geoespacial['MUNICIPIO'].str.upper()

#df_geoespacial['geometry'] = df_geoespacial['geometry'].simplify(tolerance=0.1)

In [ ]:
#unindo os dataframe df_geoespacial e df_tratado
df_bahia = pd.merge(df_geoespacial, df_tratado, left_on='MUNICIPIO', right_on='NM_MUNICIPIO', how='left')

#visualizando o df_bahia 
df_bahia.head()

In [ ]:
#verificando o número de linhas antes de excluir as linhas que não coincidem MUNICIPIO E NM_MUNICIPIO

df_bahia.shape

In [ ]:
# Identificar as linhas perdidas após o merge
linhas_perdidas = df_tratado.index.difference(df_bahia.index)

# Exibir o número de linhas perdidas e algumas amostras dessas linhas
print("Número de linhas perdidas após o merge:", len(linhas_perdidas))
print("Amostras de linhas perdidas:")
print(df_tratado.loc[linhas_perdidas].head())


# Verificar se há linhas duplicadas no DataFrame resultante
linhas_duplicadas = df_bahia[df_bahia.duplicated()].index

print("\nNúmero de linhas duplicadas no DataFrame resultante:", len(linhas_duplicadas))

In [ ]:
#contador de quantas linhas que não coincidem entre as colunas MUNICIPIO e NM_MUNICIPIO

linhas_nao_coincidentes = (df_bahia['MUNICIPIO'] != df_bahia['NM_MUNICIPIO']).sum()

print(linhas_nao_coincidentes)


In [ ]:
#visualizando as linhas onde as colunas MUNICIPIO e NM_MUNICIPIO não coincidem

linhas_nao_coincidentes = df_bahia[df_bahia['MUNICIPIO'] != df_bahia['NM_MUNICIPIO']]

municipios_nao_coincidentes = linhas_nao_coincidentes[['MUNICIPIO', 'NM_MUNICIPIO']]
print(municipios_nao_coincidentes)

In [ ]:
#excluindo as linhas onde as colunas MUNICIPIO e NM_MUNICIPIO não coincidem

df_bahia = df_bahia.drop(df_bahia[df_bahia['MUNICIPIO'] != df_bahia['NM_MUNICIPIO']].index)


In [ ]:
#verificando novamente se há linhas que não coincidem entre as colunas MUNICIPIO E NM_MUNICIPIO

linhas_nao_coincidentes = (df_bahia['MUNICIPIO'] != df_bahia['NM_MUNICIPIO']).sum()

print(linhas_nao_coincidentes)

In [ ]:
#veificando o número de linhas após a exclusão

df_bahia.shape

In [ ]:
#excluindo as colunas MUNICIPIO e Codigo, mantendo a coluna NM_MUNICIPIO
df_bahia = df_bahia.drop(columns=['MUNICIPIO', 'Codigo'])

#renomeando a coluna geometry para GEOMETRY
df_bahia = df_bahia.rename(columns={'geometry': 'GEOMETRY'})

#inserindo a coluna GEOMETRY para a posição 9 para ficar ao lado da coluna NM_MUNICIPIO
df_bahia.insert(6, 'GEOMETRY', df_bahia.pop('GEOMETRY'))


df_bahia.head()

In [ ]:
#verificando os polígonos no dataset após o tratamento

df_municipio_agrupado = df_bahia.groupby('NM_MUNICIPIO').first().reset_index()

gdf_municipio_agrupado = gpd.GeoDataFrame(df_municipio_agrupado, geometry='GEOMETRY')

gdf_municipio_agrupado.plot()

plt.show()

In [ ]:
# Contar o número de municípios únicos em cada conjunto de dados
num_municipios_bahia= df_bahia['NM_MUNICIPIO'].nunique()
num_municipios_agrupado = df_municipio_agrupado['NM_MUNICIPIO'].nunique()

print(num_municipios_bahia)
print(num_municipios_agrupado)
